# FinBERT Replication - Karsten Guenther

## Modules 

In [ ]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

In [ ]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Preparing The Model

In [ ]:
lm_path = project_dir/'models'/'languagemodel'
bm = 'bert-base-uncased'
cl_path = project_dir/'models'/'classifier_model'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring Training Parameters

In [ ]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(bm,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [ ]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [ ]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

## Fine-tune the model

In [ ]:
# Get the training examples
train_data = finbert.get_data('train')

In [ ]:
model = finbert.create_the_model()

### Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. 
Execute this step if you want a shorter training time in the expense of accuracy.

In [ ]:
# This is for fine-tuning a subset of the model.

freeze = 10

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training the model

In [ ]:
trained_model = finbert.train(train_examples = train_data, model = model)

## Testing the model

In [ ]:
test_data = finbert.get_data('test')
results = finbert.evaluate(examples=test_data, model=trained_model)

### Classification report

In [ ]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [ ]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [ ]:
report(results,cols=['labels','prediction','predictions'])